In [1]:
import time
iss = 0

def make_submit(pred):
    global iss
    submit = pd.DataFrame()
    submit['id'] = test['id']
    submit['skilled'] = pred
    iss += 1
    submit.to_csv('submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss), index=False)
    print('last: ', 'submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss))

In [2]:
import matplotlib.pyplot as plt
def print_f_imp(clf,n_top=10):
    feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
    feat_imp['feature'] = train.columns
    feat_imp.sort_values(by='importance', ascending=False, inplace=True)
    feat_imp_r = feat_imp
    feat_imp = feat_imp.iloc[:n_top]

    feat_imp.sort_values(by='importance', inplace=True)
    feat_imp = feat_imp.set_index('feature', drop=True)
    feat_imp.plot.barh(figsize=(8,8))
    
    plt.xlabel('Feature Importance Score')
    plt.show()
    return  feat_imp_r

In [3]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import lightgbm
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import catboost 
import xgboost

In [4]:
train = pd.read_csv('data/trian_v2.csv')
test = pd.read_csv('data/test_v2.csv')

##### new features

In [5]:
train = train.drop(['tm_Carry', 'tm_Escape', 'tm_Nuker', 'tm_Initiator', 'tm_Durable', 'tm_Disabler', 'tm_Jungler', 'tm_Support', 'tm_Pusher'], axis=1)

In [6]:
test = test.drop(['tm_Carry', 'tm_Escape', 'tm_Nuker', 'tm_Initiator', 'tm_Durable', 'tm_Disabler', 'tm_Jungler', 'tm_Support', 'tm_Pusher'], axis=1)

<hr>

In [7]:
train_id_damage_targets = pd.read_csv('json_data/train_id_damage_targets.csv', index_col='Unnamed: 0')

In [8]:
dtc = train_id_damage_targets.columns.tolist()

In [9]:
# new_f_c = set()
# for i in dtc:
#     new_f_c.add(i.split('_')[2])
    
# di = {}

# for i in new_f_c:
#     di[i] = []
    
# for col in dtc:
#     a = col.split('_')
#     for i in new_f_c:
#         if i in a:
#             di[i].append(col)    
            
# for i in new_f_c:
#     if len(di[i]) != 1:
#         train['dmt_' + i] = train[di[i]].sum(axis=1)

In [14]:
neutral = []
badguys = []
goodguys = []
for i  in dtc:
    a = i.split('_')
    if 'neutral' in a:
        neutral.append(i)
    if 'badguys' in a:
        badguys.append(i)
    if 'goodguys' in a:
        goodguys.append(i)

In [15]:
train['dmt_goodguys'] = train[goodguys].sum(axis=1)
train['dmt_badguys'] = train[badguys].sum(axis=1)
train['dmt_neutral'] = train[neutral].sum(axis=1)

In [16]:
test['dmt_goodguys'] = test[goodguys].sum(axis=1)
test['dmt_badguys'] = test[badguys].sum(axis=1)
test['dmt_neutral'] = test[neutral].sum(axis=1)

<hr>

In [17]:
heroes = pd.read_csv('json_data/heroes_v2.csv')

In [18]:
# train = train.merge(heroes, on='hero_id')
# test = test.merge(heroes, on='hero_id')

<hr>

In [19]:
target = train.skilled
train = train.drop(['skilled'], axis=1)

In [20]:
X = train
x_test = test
y = target

Dataset accuracy
------------ 
+ v1 - 0.8258692881849626  - lb 0.822004             
+ v1 - 0.8231688816068936 drop new features 0.8226  cv - 0.82445
+ v2 - 0.8319072759269374 - lb 
+ v2 with f - 0.8337581163905576 - lb 0.83280
<hr>
+ 0.8314521512227684 with heroes 
+ 0.8337581163905576
<hr>
+ 0.8315735178105468

In [21]:
k_fold = StratifiedKFold(n_splits=5, shuffle=True)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

In [23]:
clf =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1) #@- 82 #reg_alpha=10

In [24]:
%%time
clf.fit(X_train, y_train)

CPU times: user 13min 18s, sys: 9.96 s, total: 13min 28s
Wall time: 1min 45s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=12,
        min_child_samples=20, min_child_weight=8, min_split_gain=0.0,
        n_estimators=1500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=1, reg_alpha=9, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [25]:
accuracy_score(y_test, clf.predict(X_test))

0.8316038594574914

# Make sub

In [28]:
%%time
clf.fit(X, y)

CPU times: user 22min 59s, sys: 19.6 s, total: 23min 18s
Wall time: 3min 3s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=12,
        min_child_samples=20, min_child_weight=8, min_split_gain=0.0,
        n_estimators=1500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=1, reg_alpha=9, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [29]:
pred = clf.predict(x_test)

In [30]:
make_submit(pred)

last:  submissions/submission10-1.csv
